In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
#Import PySparkSQL
import pyspark
from pyspark.sql import SparkSession        

# Create SparkSession
spark = (SparkSession
  .builder
  .appName("SparkSQL")
  .getOrCreate())

# Path to dataset
csv_file = "/user/user1/bank-full.csv"

In [3]:
# Read and create a temporary view
df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(csv_file))
df.createOrReplaceTempView("rawdata")

In [4]:
 # Visualise inferred schema
data = spark.sql("SELECT * FROM rawdata")
data.printSchema()

root
 |-- "age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y": string (nullable = true)



In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType, StructField

# Start the SparkSession
spark = SparkSession.builder \
    .appName("Schema Correction") \
    .getOrCreate()

# Define the corrected schema
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", IntegerType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),
    StructField("contact", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("duration", IntegerType(), True),
    StructField("campaign", IntegerType(), True),
    StructField("pdays", IntegerType(), True),
    StructField("previous", IntegerType(), True),
    StructField("poutcome", StringType(), True),
    StructField("y", StringType(), True)
])

# Load the data with the corrected schema
data = spark.read \
    .option("header", "true") \
    .option("delimiter", ";") \
    .schema(schema) \
    .csv("/user/user1/bank-full.csv")

# Print the schema of the DataFrame
data.printSchema()

# Show some rows of the data
data.show(5)


root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur|married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar|married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown| single|  unknown|     no|      1|     no|  no|unknown|  5|  may|     19

In [6]:
import pyspark.sql.functions as F
import pandas as pd

In [7]:
# Função auxiliar para limpeza de texto e preenchimento de valor ausente
def preprocess_dataframe(pdf, replace_dict):
    pdf = pdf.replace(replace_dict)
    # Converter para Pandas DataFrame para utilizar fillna e dropna
    pdf = pdf.fillna(-1)
    pdf = pdf.dropna()
    return pdf

In [8]:
# Carregue o dicionário de substituição para campos categorizados
sub_dict = {'yes': 1, 'no': 0, 'unknown': -1}

In [14]:
# Expressão para a coluna 'label'
label_expression = F.when(F.col('label') == 'no', False).otherwise(True)

# Obtenha as colunas menos 'label' e 'y'
columns = df.columns[:-1]

# Crie um novo DataFrame com a coluna 'label' adicionada
df = df.select(columns + [label_expression.alias('label')])

# Coluna 'label' estará pronta para uso
df.show()

AnalysisException: cannot resolve '`label`' given input columns: ["age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"];
'Project [CASE WHEN ('label = no) THEN false ELSE true END AS label#150]
+- Relation["age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"#16] csv
